In [1]:
from fulcrum import Fulcrum
import pyautogui
import rpa 
import os
import pandas as pd
import sqlalchemy

print('Welcome, my friend.  Libraries imported.')

fulcrum = Fulcrum(key=os.environ['FULCRUM_API'])
print('\n Fulcrum API connection established.')






db_host = "postgresqlsdsu.postgres.database.azure.com"
db_name = "fac_data_warehouse"
db_user = os.environ['AZURE_PG_UKEY']
db_password = os.environ['AZURE_PG_PW']
sslmode = "require"



#Production Engine
engine = sqlalchemy.create_engine('postgresql+psycopg2://{}:{}@postgresqlsdsu.postgres.database.azure.com/fac_data_warehouse'.format(db_user, db_password))
print("Azure Connection established"+'\n' + '=================================')  





Welcome, my friend.  Libraries imported.

 Fulcrum API connection established.
Azure Connection established


In [2]:
sql='''
SELECT
room_key,
_title,
sfdb_number,
room_number,
signage_photos,
signage_photos_captions, 
door_hardware_photos,
door_hardware_photos_captions,
"360_photo",
"360_photo_captions",
room_feature_photos,
room_feature_photos_captions,
floor_cover_type,
restroom_accessible,
restroomambulatory,
restroom_accessible_transfer_side,
restroom_gender,
restroom_sink_count,
restroom_toilet_count,
restroom_urinal_count,
room_use_valid,
correct_room_use,
correct_station_count,
corrected_stations,
department_valid,
department_discrepancy_details,
correct_department,
floor_plan_valid,
floor_plan_correction_notes,
flag_as_mis_utilized,
survey_complete,
reason_not_surveyed,
not_surveyed_description,
survey_notes
FROM 
fulcrum.surveys_unprocessed
WHERE room_key IS NOT NULL;
'''.replace('\n', ' ')


print('Fetching unprocesed surveys from FDW\n')
df = pd.read_sql(sql, engine)
df['room_key'] = df['room_key'].astype('int')

print('Converting to dictionary\n')
df = df.to_dict(orient='records')

os.chdir(r'C:\Data\fmatic')

photos_to_process = ['signage_photos']

print('\n---------- Processing signage photos ----------\n')

Fetching unprocesed surveys from FDW

Converting to dictionary


---------- Processing signage photos ----------



In [28]:
df

[{'room_key': 18153,
  '_title': 'EISC-1L1, Elevator 1',
  'sfdb_number': '114',
  'room_number': '1L1',
  'signage_photos': 'e4c25161-df76-4a72-8ed7-62dffab34996,36934c63-67b1-439c-ae4a-a9b5b4e73666,2dd2fe38-2ff2-493a-8dfd-0db61d570c35',
  'signage_photos_captions': 'Elevator Controls,Evacuation Signage,Elevator Signage',
  'door_hardware_photos': None,
  'door_hardware_photos_captions': None,
  '360_photo': '216d3224-d81e-4b01-ad4f-0ce3fddc48e9',
  '360_photo_captions': None,
  'room_feature_photos': '5a1ceb81-f61f-42f3-aa66-b80f56172e78,48fbd7b5-c445-4285-8436-db650e1aeafc,47aee4e7-81bd-4801-b26d-c66050b49847',
  'room_feature_photos_captions': ',,',
  'floor_cover_type': 'Vinyl Luxury Tile',
  'restroom_accessible': None,
  'restroomambulatory': None,
  'restroom_accessible_transfer_side': None,
  'restroom_gender': None,
  'restroom_sink_count': nan,
  'restroom_toilet_count': nan,
  'restroom_urinal_count': nan,
  'room_use_valid': 'yes',
  'correct_room_use': None,
  'correct_st

In [ ]:


for i in df:
    
    if i['signage_photos'] != None:
        print('\n' + i['_title'] + ' contains ' + str(len(i['signage_photos'].split(','))) + ' photos.')

        #Check to see if there's already a dir for the given room.  If not, make it.  
        if not os.path.exists(str(i['room_key'])):
            os.mkdir(str(i['room_key']))
            os.chdir(str(i['room_key']))
            print('Created directory:' + os.getcwd())
        
            if not os.path.exists('signage_photos'):
                os.mkdir('signage_photos')
                os.chdir('./signage_photos')
                print('Signage photos directory created.')
            
            
                
        else:
            pass

        for rec in i['signage_photos'].split(','):
            
            #Get the index position of the photo
            index_pos = i['signage_photos'].split(',').index(rec)
            print('Image index position: ' + str(index_pos))

            #Add a suffix as a means of dealing with multiple photos so they don't have duplicate names
            suffix = index_pos + 1
            print('Suffix: ' + str(suffix))

            if i['signage_photos_captions'] != None and i['signage_photos_captions'] != '' :
                print('Images contain captions.')
                print(i['signage_photos_captions'].split(',')[index_pos])
                file_name =  i['_title'] + '-' + str(i['signage_photos_captions'].split(',')[index_pos])
            else:
                print('No captions.  Using default.')
                file_name = i['sfdb_number'] + '-' + i['room_number'] + '-signage-' + str(suffix)
            
            
            
            #Get the photo object from Fulcrum API
            photo = fulcrum.photos.media(rec)
            with open(f'{file_name}.jpg', 'wb') as f:
                f.write(photo)
            print(f'{file_name}.jpg' + ' saved')
            
    
        os.chdir(r'C:\Data\fmatic')
       
    